In [1]:
# Create the path (PYTHONPATH) to modules, assuming that the 'amcess' directory is in the parent directory
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
def animate_optimization(outxyz, SC_object):
    import py3Dmol

    # read calculations and convert them to a single string to plot
    water_xyz = ""
    with open(outxyz, 'r') as f:
        lines = f.readlines()
        for line in lines:
            water_xyz += line

    # 20% to check visually that everything is inside
    sr = SC_object.sphere_radius * 1.2

    # sphere center
    cx, cy, cz = SC_object.sphere_center
    sc = {"x": cx, "y": cy, "z": cz}

    # axes
    x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
    y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
    z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

    # starting visualization
    xyz_view = py3Dmol.view(width=700, height=500)#, linked=False, viewergrid=(2,2))
    xyz_view.addModelsAsFrames(water_xyz,'xyz')
    # xyz_view.setStyle({'stick': {}})
    xyz_view.setStyle({'sphere': {'radius': 0.8}})
    xyz_view.addSphere({'center': sc, 
        'radius': sr, 
        'color' :'yellow',
        'alpha': 0.5,
        })

    # animation base on several XYZ coordinates snapshots
    xyz_view.animate({'loop': "forward", 'step': 1, 'reps': 1})

    # cartesian 3D axes
    xyz_view.addLine(x_axis)
    xyz_view.addLine(y_axis)
    xyz_view.addLine(z_axis)

    xyz_view.addLabel("x", {
        'position':x_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
    xyz_view.addLabel("y", {
        'position':y_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
    xyz_view.addLabel("z", {
        'position':z_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })

    xyz_view.rotate(-15, "y", 1)
    xyz_view.rotate(-5, "z", 1)
    xyz_view.rotate(5, "x", 1)
    xyz_view.render()
    xyz_view.zoomTo()
    xyz_view.show()

    return xyz_view

In [3]:
from amcess.base_molecule import Cluster
from amcess.search_engine import SearchConfig

In [4]:
# Define a water dimer
water = {
    "atoms": [
        ("O", 0, 0, 0),
        ("H", 0.58708, 0.75754, 0),
        ("H", -0.58708, 0.75754, 0),
    ],
    "charge": 0,
    "multiplicity": 1,
}

water_dimer = Cluster(water, water).initialize_cluster(max_closeness=2.976)

In [5]:
# Define the search configuration method
cluster_opt = SearchConfig(water_dimer, search_methodology="dual_annealing", basis="6-31G(2df,p)")
cluster_opt.run(maxfun=5, maxiter=8, seed=58)



*** Cost function is Hartree--Fock implemented into pyscf ***


*** Minimization: Dual Annealing ***
Accept 2
Accept 1
Accept 2
Accept 1
Accept 2


In [6]:
xyz_opt = animate_optimization("configurations.xyz", cluster_opt)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
xyz_opt.png() # You can right click on the image to save final optimization